In [ ]:
#Imports
%matlibplot inline
from scipy import stats

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#list of players stats datafromes
lst_players_df = []
#over range 0-16 read csv into list
for i in range(17):
    if i < 10:
        lst_players_df.append(pd.read_csv('clean_players_200'+str(i)+'.csv'))
    else:
        lst_players_df.append(pd.read_csv('clean_players_20'+str(i)+'.csv'))

In [ ]:
#list of average points/pergame for each season
lst_yearly_pts_avg = []
#list of points/pergame standard deviation for each season
lst_yearly_pts_stddev = [] 
for df in lst_players_df: #for each df in list of player stat df
    lst_yearly_pts_avg.append(np.mean(df['PTS']))#calculate avg points and append into list of avgs
    lst_yearly_pts_stddev.append(np.std(df['PTS']))#calculate stddev of points and append into list of stddevs


#list of average assist/pergame for each season
lst_yearly_ast_avg = [] 
#list of assist/pergame standard deviation for each season
lst_yearly_ast_stddev = [] 
for df in lst_players_df: #for each df in list of player stat df
    lst_yearly_ast_avg.append(np.mean(df['AST']))#calculate avg assist and append into list of avgs
    lst_yearly_ast_stddev.append(np.std(df['AST']))#calculate stddev of assist and append into list of stddevs

#list of average turnover/pergame for each season
lst_yearly_tov_avg = [] 
#list of turnover/pergame standard deviation for each season
lst_yearly_tov_stddev = [] 
for df in lst_players_df: #for each df in list of player stat df
    lst_yearly_tov_avg.append(np.mean(df['TOV']))#calculate avg turnover and append into list of avgs
    lst_yearly_tov_stddev.append(np.std(df['TOV']))#calculate stddev of turnover and append into list of stddevs

#list of average rebound/pergame for each season
lst_yearly_reb_avg = [] 
#list of rebound/pergame standard deviation for each season
lst_yearly_reb_stddev = [] 
for df in lst_players_df: #for each df in list of player stat df
    lst_yearly_reb_avg.append(np.mean(df['REB']))#calculate avg rebound and append into list of avgs
    lst_yearly_reb_stddev.append(np.std(df['REB']))#calculate stddev of rebound and append into list of stddevs

#list of average steal/pergame for each season
lst_yearly_stl_avg = [] 
#list of steal/pergame standard deviation for each season
lst_yearly_stl_stddev = []
for df in lst_players_df: #for each df in list of player stat df
    lst_yearly_stl_avg.append(np.mean(df['STL']))#calculate avg steal and append into list of avgs
    lst_yearly_stl_stddev.append(np.std(df['STL']))#calculate stddev of steal and append into list of stddevs

#list of average block/pergame for each season
lst_yearly_blk_avg = [] 
#list of block/pergame standard deviation for each season
lst_yearly_blk_stddev = [] 
for df in lst_players_df: #for each df in list of player stat df
    lst_yearly_blk_avg.append(np.mean(df['BLK']))#calculate avg block and append into list of avgs
    lst_yearly_blk_stddev.append(np.std(df['BLK']))#calculate stddev of block and append into list of stddevs